In [1]:
import pandas as pd
import numpy as np
import random 
import torch 
import os 
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torch import nn
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import image
import shutil

from sklearn.mixture import GaussianMixture
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, balanced_accuracy_score

from dataloader import * 
from models import * 
from train import * 
from train_gdro import * 

ModuleNotFoundError: No module named 'train'

In [ ]:
!python3 /scratch/paa9751/mlhc-project/BalancingGroups/train-stage1-diff-loader.py --data_path /scratch/paa9751/mlhc-project/new_data --output_dir mnli_out --num_hparams_seeds 1 --num_init_seeds 1

In [ ]:
!python3 train-duplicate.py --PROJECT_NAME 'stage-1-training-sample-split' --BORDER_SZ 25 --LEARNING_RATE 1e-4 --BATCH_SIZE 128 --WEIGHT_DECAY 1e-3 --MODEL_TYPE 'resnet18' --OUTDIR out-nuisance-samplesplit --MAX_EPOCHS 200 --SPLIT_IDX 0

In [2]:
!python3 train_gdro.py --LEARNING_RATE 1e-4 --BATCH_SIZE 256 --K 10000 --MODEL_TYPE 'resnet18' --WEIGHT_DECAY 1e-3 --OUTDIR 'stage2-outdir' --GROUP_CNAME 'true_group_idx' --n_groups 4 --PROJECT_NAME 'stage-2-truegroups' --TRANSFORM

/ext3/miniconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
wandb: Currently logged in as: pooja-aryamane (suff-nuisance-stats). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /scratch/paa9751/mlhc-project/wandb/run-20240424_195644-catqbvxy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run groups_4_0.001+pretrained_False+batchsize_256+lr_0.0001_transformation_True
wandb: ⭐️ 

KeyboardInterrupt: 

In [ ]:
M_LABEL='negbio'
IMAGE_SIZE=256
TRANSFORM=False
NWORKERS=6
BATCH_SIZE=12
BORDER_SZ=0
K=10000
MAX_EPOCHS=100
LEARNING_RATE=1e-3
WEIGHT_DECAY=0
LOGFILE="./logs-ntbk"
OUTDIR="./out-ntbk"
MODEL_TYPE="resnet18"
PRETRAINED=False
RESIZE=False

In [ ]:
class NoneTransform(object):
    """ Does nothing to the image, to be used instead of None
    
    Args:
        image in, image out, nothing is done
    """
    def __call__(self, image):  
        return image

image_transformation = [NoneTransform(),
                    transforms.ElasticTransform(alpha=250.0), 
                    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)), 
                    transforms.RandomHorizontalFlip(p=0.5), 
                    transforms.RandomVerticalFlip(p=0.5), 
                    transforms.RandomRotation(degrees=(0, 180))]
trans = transforms.RandomChoice(image_transformation, [0.5, 0.1,0.1,0.1,0.1,0.1])

In [ ]:
image_transformation = [NoneTransform(),
                    transforms.ElasticTransform(alpha=250.0), 
                    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)), 
                    transforms.RandomHorizontalFlip(p=0.5), 
                    transforms.RandomVerticalFlip(p=0.5), 
                    transforms.RandomRotation(degrees=(0, 180))]
trans = transforms.RandomChoice(image_transformation, [0.5, 0.1,0.1,0.1,0.1,0.1])

In [ ]:
trans(batch_x)

In [ ]:
chexpert_dir = '/scratch/paa9751/mlhc-project/resized_data/chexpert'
mimic_dir = '/scratch/paa9751/mlhc-project/resized_data/mimic'

torch.manual_seed(42)
torch.cuda.manual_seed(42)
#     train_dataloader, val_dataloader, test_dataloader = load_combined_data(chexpert_dir, (mimic_dir, M_LABEL), IMAGE_SIZE, TRANSFORM, NWORKERS, BATCH_SIZE, BORDER_SZ, K)  

NORMALISE=True

train_dataloader, val_dataloader, test_dataloader = load_resized_data(chexpert_dir, mimic_dir, IMAGE_SIZE, False, True, NWORKERS, BATCH_SIZE, BORDER_SZ, 'true_group_idx', 0) 


In [ ]:
IMAGENET_MEAN = [0.445,0.445]         # Mean of ImageNet dataset (used for normalization)
IMAGENET_STD = [0.269,0.269]
norm = transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)

In [ ]:
for i in train_dataloader:
    path = i[0]
    batch_x = i[1]
    batch_y= i[2]
    group = i[3]
    break

In [ ]:
path[0]

In [ ]:
img_data = np.load(path[0])
img_data.shape

In [ ]:
img_tensor = torch.from_numpy(img_data)
img_tensor = img_tensor.unsqueeze(0)
img_tensor.shape

In [ ]:
(batch_x - 0.445) / 0.269

In [ ]:
# sample = batch_x[0][0]
# (sample==0).sum()



In [ ]:
sample[25:-25, 25:-25].shape

In [ ]:
(sample==0).sum()

In [ ]:
print(device)

In [ ]:
start_training_robust(model, train_dataloader, val_dataloader, device)

In [ ]:
#helper functions: 
def compute_group_avg(losses, group_idx):
    # compute observed counts and mean loss for each group
    n_groups = torch.unique(group_idx).shape[0]
    group_map = (group_idx == torch.arange(n_groups).unsqueeze(1).long()).float()
    group_count = group_map.sum(1)
    group_denom = group_count + (group_count==0).float() # avoid nans
    group_loss = (group_map @ losses.view(-1))/group_denom #check loss dim 
    return group_loss, group_count

def compute_robust_loss(group_probs, step_size, group_loss): 
    #they either normalise group loss or adjust group loss by adding adj/sqrt(count)
    group_probs = group_probs * torch.exp(step_size * group_loss)
    group_probs = group_probs/group_probs.sum()
    robust_loss = group_loss @ group_probs
    return robust_loss, group_probs.detach()

In [ ]:
M_LABEL='negbio'
IMAGE_SIZE=256
TRANSFORM=False
NWORKERS=6
BATCH_SIZE=12
BORDER_SZ=0
K=10000
MAX_EPOCHS=100
LEARNING_RATE=1e-3
WEIGHT_DECAY=0
LOGFILE="./logs-ntbk"
OUTDIR="./out-ntbk/out-erm"
MODEL_TYPE="resnet18"
PRETRAINED=False
RESIZE=False

In [ ]:
def start_training( model, train_dataloader, val_dataloader, device):
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2)
   # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
    training_losses=[]
    validation_losses=[]
    validation_accuracy=[]
    #validation_metrics={}

    auroc_metric = BinaryAUROC(thresholds=None)
    f1_metric = BinaryF1Score()
    print("training started")
    for epoch in range(MAX_EPOCHS): 
        #print('training started.')
        #train model 
        model.train()
        train_loss=0
        for batch_x, batch_y,l in train_dataloader: #16, 100, 512, 512 
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            pred = model(batch_x.float())
            pred = pred.squeeze(1)
            loss = criterion(pred, batch_y.float()) 
            train_loss+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print('iteration done')
            
        #print(f'epoch {epoch}, train done')

        train_loss = train_loss/len(train_dataloader)
        training_losses.append(train_loss)

        val_loss=0
        correct=0
        total=0
        all_preds=[]
        all_labels=[]
        
        with torch.no_grad():
            model.eval()
            sigmoid = torch.nn.Sigmoid()
            for batch_x, batch_y,l in val_dataloader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                pred = model(batch_x.float()).squeeze(1)
                predicted = (sigmoid(pred)>0.5).float()
                correct += (predicted == batch_y).sum().item() 
                total+=batch_y.size(0)
                loss = criterion(pred, batch_y.float()) 
                val_loss += loss.item()
                all_preds.append(pred.cpu())
                all_labels.append(batch_y.cpu())

            val_loss = val_loss / len(val_dataloader)
            val_acc = correct / total 
            validation_losses.append(val_loss)
            validation_accuracy.append(val_acc)
            val_f1 = f1_metric(torch.cat(all_preds,0), torch.cat(all_labels,0))
            val_auroc = auroc_metric(torch.cat(all_preds,0), torch.cat(all_labels,0))

        #if (epoch+1)%10 == 0: 
        #logging.info(f"Epoch {epoch+1} : Training loss = {train_loss}, Val loss = {val_loss}, Val Acc = {val_acc}, Val AUROC = {val_auroc}, Val F1 = {val_f1}")
        print(f"Epoch {epoch+1} : Training loss = {train_loss}, Val loss = {val_loss}, Val Acc = {val_acc}, Val AUROC = {val_auroc}, Val F1 = {val_f1}")

        if (epoch+1)%10 == 0:
            print('training works')
            torch.save(model, f'{OUTDIR}/{str(BORDER_SZ) + MODEL_TYPE + str(PRETRAINED) + str(BATCH_SIZE)}_model_{epoch+1}.pth')

        #before_lr = optimizer.param_groups[0]["lr"]
        #scheduler.step()
        #after_lr = optimizer.param_groups[0]["lr"]
        #logging.info("Epoch %d: Exponential lr %.4f -> %.4f" % (epoch, before_lr, after_lr))
    
            np.save(f'{OUTDIR}/{str(epoch+1)+str(BORDER_SZ) + str(LEARNING_RATE) + str(BATCH_SIZE)}_training_losses.npy', np.array(training_losses))
            np.save(f'{OUTDIR}/{str(epoch+1)+str(BORDER_SZ) + str(LEARNING_RATE) + str(BATCH_SIZE)}_validation_losses.npy', np.array(validation_losses))
            np.save(f'{OUTDIR}/{str(epoch+1)+str(BORDER_SZ) + str(LEARNING_RATE) + str(BATCH_SIZE)}_validation_accuracy.npy', np.array(validation_accuracy))

    torch.save(model, f'{OUTDIR}/final_model.pth')
    print('done.')

In [ ]:
start_training( model, train_dataloader, val_dataloader, device)